In [1]:
# The datetime module used to work with dates as date objects.
import datetime
# The OS module in python provides functions for interacting with the operating system.
import os
# The shutil module in Python provides many functions of high-level operations on files and collections of files.
# This module helps in automating process of copying and removal of files and directories.
import shutil

# Here we'll import data processing libraries like Numpy, Pandas and Tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf

# Import pyplot package from matplotlib library
from matplotlib import pyplot as plt
# Import keras package from tensorflow library
from tensorflow import keras

# Import Sequential function from tensorflow.keras.models
from tensorflow.keras.models import Sequential
# Import Dense, DenseFeatures function from tensorflow.keras.layers
from tensorflow.keras.layers import Dense, DenseFeatures
# Import TensorBoard function from tensorflow.keras.callbacks
from tensorflow.keras.callbacks import TensorBoard

# Here we'll show the currently installed version of TensorFlow
print(tf.__version__)
%matplotlib inline

2.1.0


In [2]:
# Defining the feature names into a list `CSV_COLUMNS`
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]
UNWANTED_COLS = ['pickup_datetime', 'key']


def features_and_labels(row_data):
# The .pop() method will return item and drop from frame. 
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label


def create_dataset(pattern, batch_size=1, mode='eval'):
# The tf.data.experimental.make_csv_dataset() method reads CSV files into a dataset
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

# The map() function executes a specified function for each item in an iterable.
# The item is sent to the function as a parameter.
    dataset = dataset.map(features_and_labels)

    if mode == 'train':
# The shuffle() method takes a sequence (list, string, or tuple) and reorganize the order of the items.
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

In [3]:
# Defining the feature names into a list `INPUT_COLS`
INPUT_COLS = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
]

# Create input layer of feature columns
feature_columns = {
    colname: tf.feature_column.numeric_column(colname)
    for colname in INPUT_COLS
    }

In [4]:
model = Sequential([
    DenseFeatures(feature_columns=feature_columns.values()),
    Dense(units=32, activation="relu", name="h1"),
    Dense(units=8, activation="relu", name="h2"),
    Dense(units=1, activation="linear", name="output")
    ])


In [5]:
# Create a custom evalution metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


# Compile the keras model
model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

# Train the model

To train your model, Keras provides three functions that can be used:

.fit() for training a model for a fixed number of epochs (iterations on a dataset).

.fit_generator() for training a model on data yielded batch-by-batch by a generator

.train_on_batch() runs a single gradient update on a single batch of data.

The .fit() function works well for small datasets which can fit entirely in memory. However, for large datasets (or if you need to manipulate the training data on the fly via data augmentation, etc) you will need to use .fit_generator() instead. The .train_on_batch() method is for more fine-grained control over training and accepts only a single batch of data.

The taxifare dataset we sampled is small enough to fit in memory, so can we could use .fit to train our model. Our create_dataset function above generates batches of training examples, so we could also use .fit_generator. In fact, when calling .fit the method inspects the data, and if it's a generator (as our dataset is) it will invoke automatically .fit_generator for training.

We start by setting up some parameters for our training job and create the data generators for the training and validation data.

We refer you the the blog post ML Design Pattern #3: Virtual Epochs for further details on why express the training in terms of NUM_TRAIN_EXAMPLES and NUM_EVALS and why, in this training code, the number of epochs is really equal to the number of evaluations we perform.

In [ ]:
TRAIN_BATCH_SIZE = 1000
NUM_TRAIN_EXAMPLES = 10000 * 5  # training dataset will repeat, wrap around
NUM_EVALS = 50  # how many times to evaluate
NUM_EVAL_EXAMPLES = 10000  # enough to get a reasonable sample

trainds = create_dataset(
    pattern='../data/t*',
    batch_size=TRAIN_BATCH_SIZE,
    mode='train')

evalds = create_dataset(
    pattern='../data/d*',
    batch_size=1000,
    mode='eval').take(NUM_EVAL_EXAMPLES//1000)

In [ ]:
%time 

steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)

LOGDIR = "./trained"
# Train the sequential model
history = model.fit(x=trainds,
                    steps_per_epoch=steps_per_epoch,
                    epochs=NUM_EVALS,
                    validation_data=evalds,
                    callbacks=[TensorBoard(LOGDIR)])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs
Train for 1 steps, validate for 10 steps
Epoch 1/50
1/1 [==============================] - 14s 14s/step - loss: 561.1435 - rmse: 23.6885 - mse: 561.1435 - val_loss: 538.9352 - val_rmse: 23.2127 - val_mse: 538.9352
Epoch 2/50
WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.460273). Check your callbacks.
1/1 [==============================] - 1s 656ms/step - loss: 501.7719 - rmse: 22.4003 - mse: 501.7719 - val_loss: 499.9065 - val_rmse: 22.3571 - val_mse: 499.9065
Epoch 3/50
1/1 [==============================] - 0s 201ms/step - loss: 463.3914 - rmse: 21.5265 - mse: 463.3914 - val_loss: 466.3172 - val_rmse: 21.5895 - val_mse: 466.3172
Epoch 4/50
1/1 [==============================] - 0s 276ms/step - loss: 430.6084 - rmse: 20.7511 - mse: 430.6084 - val_loss: 430.4976 - val_rmse: 20.7423 - val_mse: 430.4977
Epoch 5/50
1/1 [==============================] - 0s 253ms/step - loss: 417.5044 - rmse: 20.4329 - mse: 417.5044 - val_loss: 397.8781 - val_rmse: 19.9455 - val_mse: 397.8781
Epoch 6/50
1/1 [==============================] - 0s 262ms/step - loss: 345.4152 - rmse: 18.5853 - mse: 345.4152 - val_loss: 366.7722 - val_rmse: 19.1435 - val_mse: 366.7722
Epoch 7/50
1/1 [==============================] - 0s 253ms/step - loss: 363.9706 - rmse: 19.0780 - mse: 363.9706 - val_loss: 341.5944 - val_rmse: 18.4763 - val_mse: 341.5944
Epoch 8/50
1/1 [==============================] - 0s 232ms/step - loss: 289.2403 - rmse: 17.0071 - mse: 289.2403 - val_loss: 314.0481 - val_rmse: 17.7171 - val_mse: 314.0481
Epoch 9/50
1/1 [==============================] - 0s 241ms/step - loss: 286.2377 - rmse: 16.9186 - mse: 286.2377 - val_loss: 303.3531 - val_rmse: 17.4117 - val_mse: 303.3531
Epoch 10/50
1/1 [==============================] - 0s 220ms/step - loss: 257.0701 - rmse: 16.0334 - mse: 257.0701 - val_loss: 300.9174 - val_rmse: 17.3433 - val_mse: 300.9174
Epoch 11/50
1/1 [==============================] - 0s 185ms/step - loss: 271.4450 - rmse: 16.4756 - mse: 271.4450 - val_loss: 295.7931 - val_rmse: 17.1898 - val_mse: 295.7932
Epoch 12/50
1/1 [==============================] - 0s 196ms/step - loss: 250.6559 - rmse: 15.8321 - mse: 250.6559 - val_loss: 291.8766 - val_rmse: 17.0752 - val_mse: 291.8766
Epoch 13/50
1/1 [==============================] - 0s 198ms/step - loss: 272.0665 - rmse: 16.4944 - mse: 272.0665 - val_loss: 290.6524 - val_rmse: 17.0397 - val_mse: 290.6524
Epoch 14/50
1/1 [==============================] - 0s 227ms/step - loss: 256.3561 - rmse: 16.0111 - mse: 256.3561 - val_loss: 288.1296 - val_rmse: 16.9712 - val_mse: 288.1296
Epoch 15/50
1/1 [==============================] - 0s 185ms/step - loss: 250.8681 - rmse: 15.8388 - mse: 250.8681 - val_loss: 284.6306 - val_rmse: 16.8609 - val_mse: 284.6306
Epoch 16/50
1/1 [==============================] - 0s 198ms/step - loss: 298.4770 - rmse: 17.2765 - mse: 298.4770 - val_loss: 282.3497 - val_rmse: 16.7934 - val_mse: 282.3497
Epoch 17/50
1/1 [==============================] - 0s 208ms/step - loss: 232.7821 - rmse: 15.2572 - mse: 232.7821 - val_loss: 276.3227 - val_rmse: 16.6097 - val_mse: 276.3227
Epoch 18/50
1/1 [==============================] - 0s 209ms/step - loss: 233.9045 - rmse: 15.2939 - mse: 233.9045 - val_loss: 274.2226 - val_rmse: 16.5548 - val_mse: 274.2226
Epoch 19/50
1/1 [==============================] - 0s 190ms/step - loss: 241.5300 - rmse: 15.5412 - mse: 241.5300 - val_loss: 273.8148 - val_rmse: 16.5399 - val_mse: 273.8148
Epoch 20/50
1/1 [==============================] - 0s 200ms/step - loss: 231.1722 - rmse: 15.2043 - mse: 231.1722 - val_loss: 270.3832 - val_rmse: 16.4390 - val_mse: 270.3832
Epoch 21/50
1/1 [==============================] - 0s 232ms/step - loss: 244.5873 - rmse: 15.6393 - mse: 244.5873 - val_loss: 268.6810 - val_rmse: 16.3805 - val_mse: 268.6810
Epoch 22/50
1/1 [==============================] - 0s 203ms/step - loss: 245.4552 - rmse: 15.6670 - mse: 245.4552 - val_loss: 266.0358 - val_rmse: 16.3049 - val_mse: 266.0358
Epoch 23/50
1/1 [==============================] - 0s 188ms/step - loss: 232.4738 - rmse: 15.2471 - mse: 232.4738 - val_loss: 263.8402 - val_rmse: 16.2393 - val_mse: 263.8402
Epoch 24/50
1/1 [==============================] - 0s 190ms/step - loss: 221.9046 - rmse: 14.8965 - mse: 221.9046 - val_loss: 258.3069 - val_rmse: 16.0616 - val_mse: 258.3069
Epoch 25/50
1/1 [==============================] - 0s 211ms/step - loss: 217.2153 - rmse: 14.7382 - mse: 217.2153 - val_loss: 259.5989 - val_rmse: 16.1060 - val_mse: 259.5989
Epoch 26/50
1/1 [==============================] - 0s 197ms/step - loss: 251.1073 - rmse: 15.8464 - mse: 251.1073 - val_loss: 257.2844 - val_rmse: 16.0286 - val_mse: 257.2844
Epoch 27/50
1/1 [==============================] - 0s 211ms/step - loss: 204.0089 - rmse: 14.2832 - mse: 204.0089 - val_loss: 253.0699 - val_rmse: 15.8976 - val_mse: 253.0699
Epoch 28/50
1/1 [==============================] - 0s 231ms/step - loss: 218.9465 - rmse: 14.7968 - mse: 218.9465 - val_loss: 252.1912 - val_rmse: 15.8734 - val_mse: 252.1912
Epoch 29/50
1/1 [==============================] - 0s 188ms/step - loss: 245.2914 - rmse: 15.6618 - mse: 245.2914 - val_loss: 250.6115 - val_rmse: 15.8177 - val_mse: 250.6115
Epoch 30/50
1/1 [==============================] - 0s 214ms/step - loss: 221.9627 - rmse: 14.8984 - mse: 221.9627 - val_loss: 249.7443 - val_rmse: 15.7994 - val_mse: 249.7444
Epoch 31/50
1/1 [==============================] - 0s 215ms/step - loss: 216.0065 - rmse: 14.6972 - mse: 216.0065 - val_loss: 251.0478 - val_rmse: 15.8414 - val_mse: 251.0477
Epoch 32/50
1/1 [==============================] - 0s 252ms/step - loss: 227.5754 - rmse: 15.0856 - mse: 227.5754 - val_loss: 250.0740 - val_rmse: 15.8082 - val_mse: 250.0740
Epoch 33/50
1/1 [==============================] - 0s 244ms/step - loss: 230.6432 - rmse: 15.1869 - mse: 230.6432 - val_loss: 250.5666 - val_rmse: 15.8279 - val_mse: 250.5665
Epoch 34/50
1/1 [==============================] - 0s 205ms/step - loss: 217.8810 - rmse: 14.7608 - mse: 217.8810 - val_loss: 252.1180 - val_rmse: 15.8701 - val_mse: 252.1180
Epoch 35/50
1/1 [==============================] - 0s 238ms/step - loss: 209.2079 - rmse: 14.4640 - mse: 209.2079 - val_loss: 248.9013 - val_rmse: 15.7699 - val_mse: 248.9012
Epoch 36/50
1/1 [==============================] - 0s 270ms/step - loss: 266.9137 - rmse: 16.3375 - mse: 266.9137 - val_loss: 249.1861 - val_rmse: 15.7715 - val_mse: 249.1861
Epoch 37/50
1/1 [==============================] - 0s 247ms/step - loss: 239.9447 - rmse: 15.4901 - mse: 239.9447 - val_loss: 248.4395 - val_rmse: 15.7570 - val_mse: 248.4395
Epoch 38/50
1/1 [==============================] - 0s 192ms/step - loss: 220.2974 - rmse: 14.8424 - mse: 220.2974 - val_loss: 250.9961 - val_rmse: 15.8313 - val_mse: 250.9961
Epoch 39/50
1/1 [==============================] - 0s 240ms/step - loss: 203.5481 - rmse: 14.2670 - mse: 203.5481 - val_loss: 250.4058 - val_rmse: 15.8156 - val_mse: 250.4058
Epoch 40/50
1/1 [==============================] - 0s 188ms/step - loss: 207.8423 - rmse: 14.4167 - mse: 207.8423 - val_loss: 249.7776 - val_rmse: 15.7993 - val_mse: 249.7776
Epoch 41/50
1/1 [==============================] - 0s 213ms/step - loss: 199.6897 - rmse: 14.1312 - mse: 199.6897 - val_loss: 249.4542 - val_rmse: 15.7859 - val_mse: 249.4542
Epoch 42/50
1/1 [==============================] - 0s 172ms/step - loss: 224.3715 - rmse: 14.9790 - mse: 224.3715 - val_loss: 244.6346 - val_rmse: 15.6317 - val_mse: 244.6346
Epoch 43/50
1/1 [==============================] - 0s 216ms/step - loss: 211.3754 - rmse: 14.5388 - mse: 211.3754 - val_loss: 248.7197 - val_rmse: 15.7594 - val_mse: 248.7197
Epoch 44/50
1/1 [==============================] - 0s 200ms/step - loss: 208.2830 - rmse: 14.4320 - mse: 208.2830 - val_loss: 249.0528 - val_rmse: 15.7667 - val_mse: 249.0528
Epoch 45/50
1/1 [==============================] - 0s 199ms/step - loss: 237.6899 - rmse: 15.4172 - mse: 237.6899 - val_loss: 248.0569 - val_rmse: 15.7402 - val_mse: 248.0570
Epoch 46/50
1/1 [==============================] - 0s 220ms/step - loss: 216.7469 - rmse: 14.7223 - mse: 216.7469 - val_loss: 247.8345 - val_rmse: 15.7375 - val_mse: 247.8345
Epoch 47/50
1/1 [==============================] - 0s 186ms/step - loss: 262.1243 - rmse: 16.1903 - mse: 262.1243 - val_loss: 247.1873 - val_rmse: 15.7175 - val_mse: 247.1873
Epoch 48/50
1/1 [==============================] - 0s 173ms/step - loss: 212.1745 - rmse: 14.5662 - mse: 212.1745 - val_loss: 247.0517 - val_rmse: 15.7084 - val_mse: 247.0517
Epoch 49/50
1/1 [==============================] - 0s 182ms/step - loss: 210.8718 - rmse: 14.5214 - mse: 210.8718 - val_loss: 249.0218 - val_rmse: 15.7772 - val_mse: 249.0218
Epoch 50/50
1/1 [==============================] - 0s 223ms/step - loss: 243.6624 - rmse: 15.6097 - mse: 243.6624 - val_loss: 247.9750 - val_rmse: 15.7405 - val_mse: 247.9749

In [ ]:
model.summary()

Model: sequential

Layer (type)                 Output Shape              Param #
===============================================================
dense_features (DenseFeature multiple                  0
_________________________________________________________________
h1 (Dense)                   multiple                  192
_________________________________________________________________
h2 (Dense)                   multiple                  264
_________________________________________________________________
output (Dense)               multiple                  9
===============================================================
Total params: 465

Trainable params: 465

Non-trainable params: 0

In [ ]:
RMSE_COLS = ['rmse', 'val_rmse']
# The history object is returned from calls to the fit() function used to train the model.
# Metrics are stored in a dictionary in the history member of the object returned.
pd.DataFrame(history.history)[RMSE_COLS].plot()

LOSS_COLS = ['loss', 'val_loss']
# The history object is returned from calls to the fit() function used to train the model.
# Metrics are stored in a dictionary in the history member of the object returned.
pd.DataFrame(history.history)[LOSS_COLS].plot()

In [ ]:
# The predict() method will predict the response for model. 
# Using tf.convert_to_tensor() we will convert the given value to a Tensor. 
model.predict(x={"pickup_longitude": tf.convert_to_tensor([-73.982683]),
                 "pickup_latitude": tf.convert_to_tensor([40.742104]),
                 "dropoff_longitude": tf.convert_to_tensor([-73.983766]),
                 "dropoff_latitude": tf.convert_to_tensor([40.755174]),
                 "passenger_count": tf.convert_to_tensor([3.0])},
              steps=1)

array([[0.04512187]],  dtype=float32)

# Export and deploy our model
For others to use our trained model, we'll have to export our model to a file, and expect client code to instantiate the model from that exported file.

We'll export the model to a TensorFlow SavedModel format. Once we have a model in this format, we have lots of ways to "serve" the model, from a web application, from JavaScript, from mobile applications, etc.

In [ ]:
OUTPUT_DIR = "./export/savedmodel"
shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
# The join() method takes all items in an iterable and joins them into one string.
EXPORT_PATH = os.path.join(OUTPUT_DIR,
                           datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
tf.saved_model.save(model, EXPORT_PATH)  # with default serving function

WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./export/savedmodel/20200910125428/assets

In [ ]:
# Export the model to a TensorFlow SavedModel format

!saved_model_cli show \
 --tag_set serve \
 --signature_def serving_default \
 --dir {EXPORT_PATH}
!find {EXPORT_PATH}
os.environ['EXPORT_PATH'] = EXPORT_PATH